In [ ]:
# -*- coding: utf-8
# Repórter Brasil (https://ruralometro2022.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Faz o cruzamento de nomes completos e CPFs de deputados com os dados da Receita Federal para obter empresas que os deputados participam do quadro de sócios e administradores (QSA) e também os demais sócios dessas empresas e os outros CNPJs que eles possuem
# Dados - Receita (https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/dados-publicos-cnpj)
# e tabulados em SQL pelo projeto (https://github.com/rictom/cnpj-sqlite)

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
politicos = pd.read_csv("resultados/politicos_2018_09abr.csv", **kwargs)

In [3]:
politicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   nome                  510 non-null    object
 1   foto                  510 non-null    object
 2   nome_urna             510 non-null    object
 3   politico_id           510 non-null    object
 4   partido_eleicao       510 non-null    object
 5   partido_atual         510 non-null    object
 6   uri                   510 non-null    object
 7   sequencial_candidato  510 non-null    object
 8   bancada_ruralista     236 non-null    object
dtypes: object(9)
memory usage: 36.0+ KB


In [4]:
politicos['cpf_mascarado']= '***'+politicos['politico_id'].str[3:9]+'**'

In [5]:
TextFileReader = pd.read_csv('D://Code//nova_receita//socios_fev_2022.csv',\
                             chunksize=1000,\
                             sep=',',\
                             converters={'cnpj': lambda x: str(x),
                                         'cnpj_basico': lambda x: str(x)
                                        },\
                             usecols=['cnpj', 
                                      'cnpj_basico', 
                                      'identificador_de_socio', 
                                      'nome_socio', 
                                      'cnpj_cpf_socio', 
                                      'qualificacao_socio', 
                                      'pais',
                                      'representante_legal', 
                                      'nome_representante'])

In [6]:
now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    empresas_encontradas = pd.merge(politicos, \
                                 df, \
                                 left_on= ['nome','cpf_mascarado'], \
                                 right_on=['nome_socio', 'cnpj_cpf_socio'])
    
    if empresas_encontradas.empty is False:
        if conta == 0:
            df_socio1 = empresas_encontradas.copy()
        else:
            df_socio1 = pd.concat([df_socio1, empresas_encontradas])             
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)

A extração começou às:  2022-03-31 14:28:21.793560
A extração terminou às:  2022-03-31 14:30:26.331702


In [7]:
df_socio1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 758 entries, 0 to 0
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nome                    758 non-null    object 
 1   foto                    758 non-null    object 
 2   nome_urna               758 non-null    object 
 3   politico_id             758 non-null    object 
 4   partido_eleicao         758 non-null    object 
 5   partido_atual           758 non-null    object 
 6   uri                     758 non-null    object 
 7   sequencial_candidato    758 non-null    object 
 8   bancada_ruralista       402 non-null    object 
 9   cpf_mascarado           758 non-null    object 
 10  cnpj                    758 non-null    object 
 11  cnpj_basico             758 non-null    object 
 12  identificador_de_socio  758 non-null    int64  
 13  nome_socio              758 non-null    object 
 14  cnpj_cpf_socio          758 non-null    obje

In [8]:
TextFileReader = pd.read_csv('D://Code//nova_receita//empresas_fev_2022.csv',\
                             chunksize=1000,\
                             sep=',',\
                             converters={'cnpj_basico': lambda x: str(x)},\
                             usecols=['cnpj_basico', 
                                      'razao_social'])

In [9]:
now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    empresas_encontradas = pd.merge(df_socio1, \
                                 df, \
                                 left_on= 'cnpj_basico', \
                                 right_on='cnpj_basico')
    
    if empresas_encontradas.empty is False:
        if conta == 0:
            df_empresas1 = empresas_encontradas.copy()
        else:
             df_empresas1 = pd.concat([df_empresas1, empresas_encontradas])             
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)

A extração começou às:  2022-03-31 14:32:07.862947
A extração terminou às:  2022-03-31 14:36:07.583292


In [10]:
 df_empresas1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 758 entries, 0 to 0
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nome                    758 non-null    object 
 1   foto                    758 non-null    object 
 2   nome_urna               758 non-null    object 
 3   politico_id             758 non-null    object 
 4   partido_eleicao         758 non-null    object 
 5   partido_atual           758 non-null    object 
 6   uri                     758 non-null    object 
 7   sequencial_candidato    758 non-null    object 
 8   bancada_ruralista       402 non-null    object 
 9   cpf_mascarado           758 non-null    object 
 10  cnpj                    758 non-null    object 
 11  cnpj_basico             758 non-null    object 
 12  identificador_de_socio  758 non-null    int64  
 13  nome_socio              758 non-null    object 
 14  cnpj_cpf_socio          758 non-null    obje

In [11]:
TextFileReader = pd.read_csv('D://Code//nova_receita//estabelecimentos_fev_2022.csv',\
                             chunksize=1000,\
                             sep=',',\
                             converters={'cnpj_basico': lambda x: str(x),
                                         'cnpj': lambda x: str(x),
                                         'situacao_cadastral': lambda x: str(x),
                                         'cnae_fiscal': lambda x: str(x),
                                         'cnae_fiscal_secundaria': lambda x: str(x),
                                         'municipio': lambda x: str(x)
                                        },\
                             usecols=['cnpj_basico', 
                                      'cnpj', 
                                      'nome_fantasia', 
                                      'situacao_cadastral', 
                                      'cnae_fiscal', 
                                      'cnae_fiscal_secundaria', 
                                      'uf', 
                                      'municipio'])

In [12]:
now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    empresas_encontradas = pd.merge(df_empresas1, \
                                 df, \
                                 left_on= 'cnpj', \
                                 right_on='cnpj')
    
    if empresas_encontradas.empty is False:
        if conta == 0:
            df_empresas_final = empresas_encontradas.copy()
        else:
             df_empresas_final = pd.concat([df_empresas_final, empresas_encontradas])             
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)

A extração começou às:  2022-03-31 14:36:36.602095
A extração terminou às:  2022-03-31 14:43:23.328621


In [13]:
 df_empresas_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 758 entries, 0 to 0
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nome                    758 non-null    object 
 1   foto                    758 non-null    object 
 2   nome_urna               758 non-null    object 
 3   politico_id             758 non-null    object 
 4   partido_eleicao         758 non-null    object 
 5   partido_atual           758 non-null    object 
 6   uri                     758 non-null    object 
 7   sequencial_candidato    758 non-null    object 
 8   bancada_ruralista       402 non-null    object 
 9   cpf_mascarado           758 non-null    object 
 10  cnpj                    758 non-null    object 
 11  cnpj_basico_x           758 non-null    object 
 12  identificador_de_socio  758 non-null    int64  
 13  nome_socio              758 non-null    object 
 14  cnpj_cpf_socio          758 non-null    obje

In [14]:
 df_empresas_final.to_excel('resultados/empresas_deputados/empresas_deputados_fev_2022.xlsx',sheet_name='Sheet1',index=False)

In [23]:
df_empresas_deputados = df_empresas_final[['nome', 'nome_urna', 'politico_id', 'cpf_mascarado', 'cnpj', 'cnpj_basico_x', 'nome_socio', 'cnpj_cpf_socio', 'razao_social', 'nome_fantasia', 'situacao_cadastral', 'cnae_fiscal', 'cnae_fiscal_secundaria', 'uf', 'municipio']].copy()

In [24]:
df_empresas_deputados.rename(columns = {'cnpj_basico_x':'cnpj_raiz'},inplace = True)

In [26]:
df_empresas_deputados.to_csv('resultados/empresas_deputados/todas_empresas_deputados_fev_2022.csv',index=False)

In [15]:
df_empresas_limpo =  df_empresas_final[['cnpj', 'cnpj_basico_x']]

In [16]:
TextFileReader = pd.read_csv('D://Code//nova_receita//socios_fev_2022.csv',\
                             chunksize=1000,\
                             sep=',',\
                             converters={'cnpj': lambda x: str(x),
                                         'cnpj_basico': lambda x: str(x)
                                        },\
                             usecols=['cnpj', 
                                      'cnpj_basico', 
                                      'identificador_de_socio', 
                                      'nome_socio', 
                                      'cnpj_cpf_socio', 
                                      'qualificacao_socio', 
                                      'pais',
                                      'representante_legal', 
                                      'nome_representante'])

In [17]:
now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    empresas_encontradas = pd.merge(df_empresas_limpo, \
                                 df, \
                                 left_on= 'cnpj', \
                                 right_on='cnpj')
    
    if empresas_encontradas.empty is False:
        if conta == 0:
            df_socio1 = empresas_encontradas.copy()
        else:
            df_socio1 = pd.concat([df_socio1, empresas_encontradas])             
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)

A extração começou às:  2022-03-31 14:59:33.626637
A extração terminou às:  2022-03-31 15:01:48.647273


In [18]:
df_socio1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2101 entries, 0 to 11
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cnpj                    2101 non-null   object 
 1   cnpj_basico_x           2101 non-null   object 
 2   cnpj_basico             2101 non-null   object 
 3   identificador_de_socio  2101 non-null   int64  
 4   nome_socio              2101 non-null   object 
 5   cnpj_cpf_socio          2099 non-null   object 
 6   qualificacao_socio      2101 non-null   int64  
 7   pais                    3 non-null      float64
 8   representante_legal     2101 non-null   object 
 9   nome_representante      88 non-null     object 
dtypes: float64(1), int64(2), object(7)
memory usage: 180.6+ KB


In [27]:
df_empresas_deputados_socios = df_socio1[['cnpj', 'cnpj_basico', 'nome_socio', 'cnpj_cpf_socio']].copy()
df_empresas_deputados_socios.rename(columns = {'cnpj_basico':'cnpj_raiz'},inplace = True)

In [28]:
df_empresas_deputados_socios.to_csv('resultados/empresas_deputados/todos_socios_empresas_deputados_fev_2022.csv',index=False)

In [19]:
df_socio1.to_excel('resultados/empresas_deputados/todos_socios_empresas_deputados_fev_2022.xlsx',sheet_name='Sheet1',index=False)